# Category Theory for Programmers

## Chapter 1: Category - The Essence of Composition

### Challenge 1

In [ ]:
def id[A](a: A): A = a

### Challenge 2

In [ ]:
def comp[A, B, C](f: B => C, g: A => B): A => C = x => f(g(x))

### Challenge 3

In [ ]:
def testCompId[A, B](f: A => B)(x: A): Boolean = {
    val y = f(x)
    (comp(f, id[A])(x) == y) && (y == comp(id[B], f)(x))
}

In [ ]:
testCompId[Integer, String](_.toString)(42)

## Chapter 2: Types and Functions

### Challenge 1

In [ ]:
import scala.collection.mutable.Map

def memoize[A, B](f: A => B): A => B = {
    val cache = Map[A, B]()
    x => cache get x match {
        case Some(y) => y
        case None => {
            val y = f(x)
            cache += (x -> y)
            y
        }
    }
}

In [ ]:
val addOne: (Integer => Integer) = memoize(_ + 1)

In [ ]:
addOne(41)

In [ ]:
addOne(41)

### Challenge 2

In [ ]:
import scala.util.Random

implicit val rnd = new Random

def rand(implicit rnd: Random): Int = {
    rnd.nextInt
}

In [ ]:
rand

In [ ]:
rand

In [ ]:
val cachedRand = memoize[Unit, Int](_ => rand)

In [ ]:
cachedRand()

In [ ]:
cachedRand()

### Challenge 3

In [ ]:
def seededRand(seed: Int): Int = {
    val rnd = new Random(seed)
    rnd.nextInt
}

val cachedSeededRand = memoize[Int, Int](seededRand)

In [ ]:
cachedSeededRand(42)

In [ ]:
cachedSeededRand(42)

## Chapter 4: Kleisli Categories

### Challenge 2 & 3

In [ ]:
import scala.math.sqrt

def safe_root(x: Double): Option[Double] = if (x >= 0) Some(sqrt(x)) else None

In [ ]:
safe_root(-1)

In [ ]:
safe_root(9)

In [ ]:
def safe_recip(x: Double): Option[Double] = if (x != 0) Some(1 / x) else None

In [ ]:
safe_recip(0)

In [ ]:
safe_recip(2)

In [ ]:
def safe_root_recip(x: Double): Option[Double] = for {
    y <- safe_recip(x)
    z <- safe_root(y)
} yield z

In [ ]:
safe_root_recip(-1)

In [ ]:
safe_root_recip(0)

In [ ]:
safe_root_recip(9)

## Chapter 5: Products and Coproducts

### Challenge 4

In [ ]:
sealed trait MyEither[A, B]

case class MyLeft[A](left: A) extends MyEither[A, Nothing]
case class MyRight[B](right: B) extends MyEither[Nothing, B]

### Challenge 5
Show that `Either` is "better" *coproduct* than `Int` with two injections `i` and `j`.

In [ ]:
def i(n: Int): Int = n
def j(b: Boolean): Int = if (b) 0 else 1

In [ ]:
def factorizer[A, B, C]: (A => C) => (B => C) => Either[A, B] => C = i => j => {
    case Left(a) => i(a)
    case Right(b) => j(b)
}

// m factorizes i and j => Either is "better" than Int
def m(e: Either[Int, Boolean]): Int = factorizer(i)(j)(e)

## Chapter 6: Simple Algebraic Data Types

### Challenge 2, 3 & 4
Define `Shape` and associated methods `area` and `circ` in OOP stype, then add new shape `Square`.

In [ ]:
import scala.math.Pi

trait Shape {
    def area: Double
    def circ: Double
}

class Circle(r: Double) extends Shape {
    def area = Pi * this.r * this.r
    def circ = 2 * Pi * this.r
}

class Rect(d: Double, h: Double) extends Shape {
    def area = this.d * this.h
    def circ = 2 * (this.d + this.h)
}

class Square(s: Double) extends Shape {
    def area = this.s * this.s
    def circ = 4 * this.s
}

In OOP the ADT pattern matching is replaced by dynamic dispatch via a *vtable*. The vtable basically keeps track of "type labels" sub-classes.

When addin new methods to a class hierarchy, one has to change every class and the trait, whereas with functions defined on ADTs, one simply adds new function.

In [ ]:
sealed trait Shape

case class Circle(r: Double) extends Shape
case class Rect(d: Double, h: Double) extends Shape
case class Square(s: Double) extends Shape

def area[S <: Shape](s: S): Double = s match {
    case Circle(r) => Pi * r * r
    case Rect(d, h) => d * h
    case Square(s) => s * s
}

def circ[S <: Shape](s: S): Double = s match {
    case Circle(r) => 2 * Pi * r
    case Rect(d, h) => 2 * (d + h)
    case Square(s) => 4 * s
}

## Chapter 7: Functors

### Challenge 3
Implemnt the *Reader* functor.

```haskell
instance Functor ((->) r) where
    -- fmap :: (a -> b) -> (r -> a) -> (r -> b)
    fmap f g = f . g
```

The solution requires the [Kind Projector](https://github.com/typelevel/kind-projector) compiler plugin for the `R => ?` type.

```scala
trait Functor[F[_]] {
    def fmap[A, B](f: A => B)(fa: F[A]): F[B]
}

implicit def readerFunctor[R] = new Functor[R => ?] {
    def fmap[A, B](f: A => B)(fa: R => A): R => B = f compose fa
}
```

## Chapter 8: Functoriality

### Challenge 5
Give an instance of a *bifunctor* for generic pair.

In [ ]:
import scala.Tuple2

trait Bifunctor[F[_, _]] {
    
    def bimap[A, B, C, D](g: A => B)(h: C => D): F[A, C] => F[B, D] = first(g) compose second(h)
    
    def first[A, B, C](g: A => C): F[A, B] => F[C, B] = bimap(g)(identity[B])
    
    def second[A, B, D](h: B => D): F[A, B] => F[A, D] = bimap(identity[A])(h)
}

implicit val pairBF = new Bifunctor[Tuple2] {
    override def bimap[A, B, C, D](g: A => B)(h: C => D): ((A, C)) => (B, D) = {
        case (x, y) => (g(x), h(y))
    }
}

In [ ]:
def tryPairBF(p: (Int, Int))(implicit BF: Bifunctor[Tuple2]): (Int, Int) = 
    BF.bimap[Int, Int, Int, Int](_ + 1)(x => x * x)(p)

In [ ]:
tryPairBF((41, 3))